# 03 — Configurer **SSD MobileNet V3 Small** (1 classe : *frelon*)

Nous allons utiliser un pipeline TFOD basé sur **SSD MobileNet V3 Small FPNLite 320x320** et l'adapter à **1 classe**.

## 1) Téléchargement du checkpoint & du pipeline

In [ ]:
import os

# Répertoires
PIPELINE_DIR = "training"
os.makedirs(PIPELINE_DIR, exist_ok=True)

# Téléchargez un modèle de base depuis le Model Zoo (ex. ssd_mobilenet_v3_small_fpnlite_320x320)
# Lien: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
# Exemple (à lancer dans un terminal si wget/curl):
# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v3_small_fpnlite_320x320_coco17_tpu-8.tar.gz
# !tar -xvf ssd_mobilenet_v3_small_fpnlite_320x320_coco17_tpu-8.tar.gz -C {PIPELINE_DIR}
print("Placez le dossier du modèle dézippé dans:", os.path.abspath(PIPELINE_DIR))

## 2) Éditer le `pipeline.config` pour 1 classe

In [ ]:
from pathlib import Path
import re

# Indiquez ces chemins selon votre organisation
PIPELINE_CONFIG_PATH = Path(PIPELINE_DIR) / "ssd_mobilenet_v3_small_fpnlite_320x320_coco17_tpu-8" / "pipeline.config"
TF_RECORDS_DIR = "data/tfrecords_single_class"
LABEL_MAP_PATH = "data/label_map.pbtxt"
MODEL_DIR = "checkpoints/ssd_mobilenet_v3_small_frelon"

os.makedirs(os.path.dirname(MODEL_DIR), exist_ok=True)

# Créer un label_map à 1 classe
label_map = """
item {
  id: 1
  name: "frelon"
}
"""
os.makedirs(os.path.dirname(LABEL_MAP_PATH), exist_ok=True)
with open(LABEL_MAP_PATH, "w", encoding="utf-8") as f:
    f.write(label_map.strip())

print("LABEL_MAP_PATH créé:", os.path.abspath(LABEL_MAP_PATH))

# Helper: fonction simple pour remplacer des paramètres (si pipeline existe)
def replace_in_file(path, patterns):
    txt = Path(path).read_text(encoding="utf-8")
    for k,v in patterns.items():
        txt = re.sub(k, v, txt)
    Path(path).write_text(txt, encoding="utf-8")

if PIPELINE_CONFIG_PATH.exists():
    print("Édition de:", PIPELINE_CONFIG_PATH)
    replacements = {
        r"num_classes: \d+": "num_classes: 1",
        r"fine_tune_checkpoint_type: "classification"": "fine_tune_checkpoint_type: "detection"",
    }
    replace_in_file(PIPELINE_CONFIG_PATH, replacements)

    # Injecter chemins (simplifié — vous pouvez aussi éditer manuellement le fichier)
    pc = Path(PIPELINE_CONFIG_PATH).read_text(encoding="utf-8")
    pc = pc.replace("PATH_TO_BE_CONFIGURED/model.ckpt", str(Path(PIPELINE_DIR)/"ssd_mobilenet_v3_small_fpnlite_320x320_coco17_tpu-8"/"checkpoint"/"ckpt-0"))
    pc = pc.replace("PATH_TO_BE_CONFIGURED/label_map", LABEL_MAP_PATH)
    pc = pc.replace("PATH_TO_BE_CONFIGURED/train.record*", str(Path(TF_RECORDS_DIR)/"train-*.record"))
    pc = pc.replace("PATH_TO_BE_CONFIGURED/val.record*",   str(Path(TF_RECORDS_DIR)/"val-*.record"))
    Path(PIPELINE_CONFIG_PATH).write_text(pc, encoding="utf-8")
    print("Chemins mis à jour dans pipeline.config")
else:
    print("⚠️ pipeline.config introuvable. Placez d'abord le modèle dans", PIPELINE_DIR)